In [ ]:
# 🧪 Install required packages (quiet mode)
!pip install -q requests matplotlib pandas pyinputplus ipywidgets

# --- IMPORTS ---
import requests
import matplotlib.pyplot as plt
import pandas as pd
import datetime
import re
import pyinputplus as pyip
from typing import Dict, Any
import ipywidgets as widgets
from IPython.display import display, clear_output

# --- GEOLOCATION FUNCTION ---
def geocode_location(location: str) -> tuple[float, float]:
    # Converts a location name to latitude and longitude using Nominatim API.
    url = "https://nominatim.openstreetmap.org/search"
    params = {'q': location, 'format': 'json', 'limit': 1}
    headers = {'User-Agent': 'weather-colab'}  # Required by Nominatim API
    response = requests.get(url, params=params, headers=headers)
    response.raise_for_status()  # Raise error if request fails
    data = response.json()
    if not data:
        raise ValueError(f"❌ Location '{location}' not found.")
    # Return the first result's latitude and longitude as floats
    return float(data[0]['lat']), float(data[0]['lon'])

# --- FETCH WEATHER DATA FUNCTION ---
def fetch_weather(lat: float, lon: float, start_date: str, end_date: str) -> Dict[str, Any]:
    # Fetches weather data from Open-Meteo API for the given coordinates and date range.
    url = "https://api.open-meteo.com/v1/forecast"
    params = {
        'latitude': lat,
        'longitude': lon,
        'daily': 'temperature_2m_max,temperature_2m_min,precipitation_sum',
        'current_weather': True,                                                 # Include current weather conditions
        'timezone': 'auto',                                                      # Automatically adjust timezone based
        'start_date': start_date,
        'end_date': end_date
    }
    return requests.get(url, params=params).json()                   # Send GET request to the API with parameters and return the JSON response as a dict

# --- FORECAST DATA EXTRACTION FUNCTION ---
def _extract_forecast_data(weather_data, keys):
    # Utility: Extracts lists of values for specified keys from weather_data['forecast'].
    forecast = weather_data.get('forecast', [])
    return [[day.get(k, None) for day in forecast] for k in keys]    # For each key, build a list of its values from each day in the forecast

# --- MAIN WEATHER DATA FUNCTION ---
def get_weather_data(location: str, forecast_days: int = 5) -> Dict[str, Any]:
    # Gets weather data for a location for up to 5 days.
    forecast_days = min(max(forecast_days, 1), 5)                    # Clamp days between 1 and 5
    lat, lon = geocode_location(location)                            # Convert location name to coordinates
    today = datetime.date.today()
    end_date = today + datetime.timedelta(days=forecast_days - 1)
    raw_data = fetch_weather(lat, lon, today.isoformat(), end_date.isoformat())
    # Structure the result
    result = {
        'location': location,
        'latitude': lat,
        'longitude': lon,
        'current': raw_data.get('current_weather', {}),             # Current weather conditions
        'forecast': []
    }
    daily = raw_data.get('daily', {})                               # Iterate through daily forecast data and build a list of daily weather dicts
    # Build the forecast list: each entry is a dict for a day's weather
    for date, t_max, t_min, precip in zip(
        daily.get('time', []),
        daily.get('temperature_2m_max', []),
        daily.get('temperature_2m_min', []),
        daily.get('precipitation_sum', [])
    ):
        result['forecast'].append({
            'date': date,
            'temp_max': t_max,
            'temp_min': t_min,
            'precipitation': precip
        })
    return result

# --- TEMPERATURE VISUALIZATION FUNCTION ---
def create_temperature_visualisation(weather_data, output_type='display'):
    # Plots max and min temperatures as a line chart.
    dates, temp_max, temp_min = _extract_forecast_data(weather_data, ['date', 'temp_max', 'temp_min'])     # Extract dates, max temps, and min temps from the weather data
    fig, ax = plt.subplots(figsize=(10, 5))                                                                # Create a matplotlib figure and axis
    ax.plot(dates, temp_max, label='Max Temp (°C)', marker='o', color='crimson')                           # Plot max temperature as a red line with circle markers
    ax.plot(dates, temp_min, label='Min Temp (°C)', marker='o', color='royalblue')
    ax.fill_between(dates, temp_min, temp_max, color='lightgray', alpha=0.3)               # Fill the area between min and max temperature lines with light gray shading
    ax.set_title(f"Temperature Forecast for {weather_data.get('location', 'Unknown')}")    # Set the plot title with the location name
    ax.set_xlabel("Date")                                                                  # Label the x-axis and y-axis
    ax.set_ylabel("Temperature (°C)")
    ax.legend()
    ax.grid(True, linestyle='--', alpha=0.5)
    plt.xticks(rotation=45)
    plt.tight_layout()                                                                      # Adjust layout to prevent clipping of labels
    if output_type == 'figure':                                                             # Return figure or display plot based on output_type
        return fig
    else:
        plt.show()

# --- PRECIPITATION VISUALIZATION FUNCTION ---
def create_precipitation_visualisation(weather_data, output_type='display'):
    # Plots precipitation as a bar chart.
    forecast = weather_data.get('forecast', [])
    if forecast:
        # Extract dates and precipitation, defaulting missing precipitation to 0
        dates, precipitation = zip(*[
            (day.get('date', 'Unknown'), day.get('precipitation') if day.get('precipitation') is not None else 0)
            for day in forecast if 'date' in day
        ])
    else:
        dates, precipitation = ([], [])
    fig, ax = plt.subplots(figsize=(10, 5))                               # Create a new figure and axis object for the plot with a size of 10x5 inches

    # Plot the precipitation data as a bar chart
    ax.bar(dates, precipitation, color='deepskyblue', width=0.6)          # - Bars colored in 'deepskyblue' for visual appeal
    ax.set_title(f"Precipitation Forecast for {weather_data.get('location', 'Unknown')}")
    ax.set_xlabel("Date")                                                 # Label the x-axis as 'Date'
    ax.set_ylabel("Precipitation (mm)")                                   # Label the y-axis as 'Precipitation (mm)'
    ax.grid(axis='y', linestyle='--', alpha=0.5)
    plt.xticks(rotation=45)
    plt.tight_layout()
    if output_type == 'figure':
        return fig
    else:
        plt.show()

# --- NATURAL LANGUAGE QUESTION PARSING ---
def parse_weather_question(question):
    # Parses a user's question to extract location, time period, and weather attribute.
    original = question                                 # Keep original question for case-sensitive operations later
    question = question.lower()                         # Convert to lowercase for case-insensitive matching
    location = None                                     # Location not yet identified
    time_period = 'today'
    weather_attribute = 'general'
    is_vague = False                                     # Flag to mark if question lacks clarity

    # Detect time period from keywords
    if 'tomorrow' in question:
        time_period = 'tomorrow'                         # User explicitly asked about tomorrow
    elif 'soon' in question or 'later' in question:
        time_period = 'next_1_days'                      # Vague future references interpreted as next 1 day
    else:
        match = re.search(r'next (\d+) days', question)  # Look for pattern like "next 3 days" to extract specific day count
        if match:
            time_period = f"next_{match.group(1)}_days"

    # Detect weather attribute from keywords
    if any(term in question for term in ['temperature', 'temp', 'hot', 'cold']):       # Check if the question contains any terms related to temperature
        weather_attribute = 'temperature'
    elif any(term in question for term in ['rain', 'precipitation', 'snow', 'wet']):   # Check if the question contains any terms related to precipitation or rain
        weather_attribute = 'precipitation'       # Set attribute to precipitation
    elif 'forecast' in question:                  # Check if the question mentions a general forecast
        weather_attribute = 'forecast'            # Set attribute to general forecast
    elif 'humidity' in question:
        weather_attribute = 'humidity'
    elif 'wind' in question:
        weather_attribute = 'wind'

    # Try to extract location: first look for "in <location>", else capitalized words
    location_match = re.search(r'\bin ([a-z\s\-]+)', question)
    if location_match:                                         # If the pattern is found, extract the location string and remove any leading/trailing whitespace
        location = location_match.group(1).strip()
    else:
        # fallback: find capitalized words as location
        caps = re.findall(r'\b[A-Z][a-z]+(?:\s[A-Z][a-z]+)?', original)
        if caps:
            location = ' '.join(caps)                         # Join multiple capitalized words to form multi-word location names

    # Mark as vague if missing key info
    if not location or weather_attribute == 'general':
        is_vague = True

    return {                                              # Return a dictionary with the extracted location, time period, weather attribute, and vagueness flag
        'location': location,
        'time_period': time_period,
        'weather_attribute': weather_attribute,
        'is_vague': is_vague
    }

# --- GENERATE WEATHER RESPONSE ---
def generate_weather_response(parsed: dict, weather_data: dict) -> str:
    # Returns a human-readable response based on the parsed question and weather data.
    location = parsed['location']
    time = parsed['time_period']
    attr = parsed['weather_attribute']
    forecast = weather_data.get('forecast', [])                                  # List of daily forecasts
    current = weather_data.get('current', {})                                    # Current weather data

    if parsed['is_vague']:                                                       # If the question is vague (missing location or attribute), prompt user for more info
        return "❗ Please include a location and weather detail (e.g. temperature or rain)."

    # Helper: find forecast for a specific date
    def forecast_for_date(date_str):
        for day in forecast:
            if day['date'] == date_str:
                return day
        return None

    today = datetime.date.today()                                            # Determine the target date based on the requested time period
    if time == 'today':
        target_date = today.isoformat()
    elif time == 'tomorrow':
        target_date = (today + datetime.timedelta(days=1)).isoformat()
    elif time.startswith('next_'):
        # For multi-day forecasts, just return summary
        days = int(time.split('_')[1])                                      # For multi-day forecasts, extract the number of days requested
        temps = [f"{d['date']}: {d['temp_min']}°C - {d['temp_max']}°C" for d in forecast[:days]]
        return "🌡️ Temperature forecast:\n" + "\n".join(temps)             # Return the multi-day temperature forecast as a formatted string
    else:
        target_date = today.isoformat()

    day_forecast = forecast_for_date(target_date)                          # Retrieve the forecast for the target date
    if not day_forecast:
        return f"⚠️ No forecast data available for {target_date}."

    # Handle precipitation (rain/snow)
    if attr == 'precipitation':                                            # Precipitation (rain, snow, wetness)
        precip = day_forecast.get('precipitation', 0)
        if precip > 0:
            return f"🌧️ Yes, it looks like there will be precipitation ({precip} mm) in {location} on {target_date}."
        else:
            return f"❌ No, it doesn't look like it will rain or snow in {location} on {target_date}."

    # Handle temperature
    elif attr == 'temperature':                                             # Temperature range
        return (f"🌡️ On {target_date} in {location}, the temperature will range from "
                f"{day_forecast['temp_min']}°C to {day_forecast['temp_max']}°C.")
    # Handle wind
    elif attr == 'wind':
        windspeed = current.get('windspeed', 'unknown')                     # Wind speed (from current weather data)
        return f"💨 Wind speed in {location} is {windspeed} km/h."
    # Handle humidity (not available)
    elif attr == 'humidity':                                                # Humidity (not available in this API)
        return "💧 Sorry, humidity data is not available from this API."

    # Generic fallback
    return "⚠️ Could not understand your weather request."  # Fallback response if attribute is unrecognized

# --- IPYWIDGETS UI INTEGRATION ---

# Widgets for user input
location_input = widgets.Text(
    description='📍 Location:',                            # Label shown next to the input box
    placeholder='e.g., Sydney',                            # Placeholder text inside the input box
    style={'description_width': 'initial'}                 # Style to control label width for better layout
)

# Text input widget for the user to enter a natural language weather question
question_input = widgets.Text(
    description='💬 Ask:',
    placeholder='e.g., Will it rain in Berlin tomorrow?',    # Placeholder text
    style={'description_width': 'initial'}                   # Style for label width
)

# Custom colored buttons
ask_button = widgets.Button(
    description='Ask Weather',                 # Button text
    button_style='',                           # No predefined style
    layout=widgets.Layout(width='150px')
)
ask_button.style.button_color = '#3498db'      # Blue

temp_button = widgets.Button(
    description='Show Temperature',
    button_style='success',  # Green
    layout=widgets.Layout(width='150px')      # Fixed width for consistent UI
)
# No custom color needed; 'success' is green

precip_button = widgets.Button(
    description='Show Precipitation',         # Button text
    button_style='',                          # No predefined style
    layout=widgets.Layout(width='180px')
)
precip_button.style.button_color = '#f39c12'  # Orange

output_area = widgets.Output()

# --- HANDLERS FOR BUTTONS ---
def on_ask_clicked(b):
    # Handles when the user clicks "Ask Weather"
    with output_area:
        clear_output()                                     # Clear previous output to keep display clean
        question = question_input.value.strip()            # Get the question text, trimmed of whitespace
        parsed = parse_weather_question(question)
        if not parsed['location']:
            print("❌ Please specify a location.")         # If no location was found in the question, prompt the user to specify one
            return
        try:
            data = get_weather_data(parsed['location'])
            print(generate_weather_response(parsed, data)) # Generate and print a user-friendly weather response based on parsed question and data
        except Exception as e:                             # If an error occurs (e.g., invalid location, network issue), print an error message
            print(f"⚠️ {e}")

def on_temp_clicked(b):
    # Handles when the user clicks "Show Temperature"
    with output_area:
        clear_output()
        loc = location_input.value.strip()     # Get the location input from the user, removing extra spaces
        if not loc:
            print("❌ Enter a location.")     # If no location is entered, show an error message and exit the function
            return
        try:
            data = get_weather_data(loc)
            create_temperature_visualisation(data)     # Create and display the temperature visualization using the fetched data
        except Exception as e:
            print(f"⚠️ {e}")                           # If any error occurs (e.g., invalid location or network issue), display an error message

def on_precip_clicked(b):
    # Handles when the user clicks "Show Precipitation"
    with output_area:
        clear_output()                        # Clear previous output to keep the display clean
        loc = location_input.value.strip()    # Get the location input from the user, trimming whitespace
        if not loc:
            print("❌ Enter a location.")     # If no location is entered, show an error message and exit the function
            return
        try:
            data = get_weather_data(loc)      # Attempt to fetch weather data for the entered location
            create_precipitation_visualisation(data)
        except Exception as e:                # If any error occurs (e.g., invalid location or network issue), display an error message
            print(f"⚠️ {e}")

# Link buttons to their handlers
ask_button.on_click(on_ask_clicked)           # Connect "Ask Weather" button to its handler
temp_button.on_click(on_temp_clicked)         # Connect "Show Temperature" button to its handler
precip_button.on_click(on_precip_clicked)     # Connect "Show Precipitation" button to its handler

# --- LAYOUT AND DISPLAY ---
ui = widgets.VBox([
    widgets.HTML("<h2>☁️ Weather Assistant</h2>"),    # HTML widget to display a heading for the app with a cloud emoji
    question_input,                                   # Text input widget where users can type their natural language weather question
    ask_button,                                       # Button widget for submitting the weather question ("Ask Weather")
    widgets.HTML("<hr>"),
    location_input,
    widgets.HBox([temp_button, precip_button]),      # Horizontal box (HBox) layout to place the temperature and precipitation buttons side by side
    output_area                                      # Output widget area to display results, messages, or plots generated by the app
])

# Display the interactive UI in the notebook
display(ui)
